# STA 141B Project Group 12

## Introduction

## Methods

In [1]:
import requests
import pandas as pd
import requests_cache
import time
requests_cache.install_cache("Final_Project")

In [59]:
url = 'https://www.cheapshark.com/api/1.0/deals' # Testing environment

result = requests.get(url, params= {
    'storeID' : 1,
    'AAA' : 1,
    'steamAppID' : 168162
})


In [64]:
test_set = pd.DataFrame()
for i in range(1, 2):
    result = requests.get(url, params= {
    'storeID' : 1,
    'pageNumber' : i,
    'AAA' : 1
    })

    data = result.json()
    data = pd.DataFrame(data)

    test_set = pd.concat([test_set, data], ignore_index=True)

    time.sleep(2)

remove later (rel code below)

In [65]:
full_set = pd.DataFrame()
for i in range(1, 51):
    result = requests.get(url, params= {
    'storeID' : 1,
    'pageNumber' : i
    })

    data = result.json()
    data = pd.DataFrame(data)

    full_set = pd.concat([full_set, data], ignore_index=True)

    time.sleep(2)

full_set = full_set[["title", "gameID", "releaseDate", "normalPrice", "metacriticScore", "steamRatingText", "steamRatingPercent"]]

In [53]:
aaa_set = pd.DataFrame()
for i in range(1, 46):
    result = requests.get(url, params= {
    'storeID' : 1,
    'pageNumber' : i,
    'AAA' : 1
    })

    data = result.json()
    data = pd.DataFrame(data)

    aaa_set = pd.concat([aaa_set, data], ignore_index=True)

    time.sleep(2)

aaa_set = aaa_set[["title", "gameID", "releaseDate", "normalPrice", "metacriticScore", "steamRatingText", "steamRatingPercent"]]

In [73]:
aaa_set["normalPrice"] = aaa_set["normalPrice"].astype(float)
aaa_set["metacriticScore"] = aaa_set["metacriticScore"].astype(int)
aaa_set["steamRatingPercent"] = aaa_set["steamRatingPercent"].astype(int)

In [71]:
full_set["normalPrice"] = full_set["normalPrice"].astype(float)
full_set["metacriticScore"] = full_set["metacriticScore"].astype(int)
full_set["steamRatingPercent"] = full_set["steamRatingPercent"].astype(int)
type(full_set["normalPrice"][1])

numpy.float64

In [72]:
#aaa_set = full_set[full_set["normalPrice"] >= 29]
indie_set = full_set[full_set["normalPrice"] < 29]
indie_set = indie_set[indie_set["metacriticScore"] > 0] # To account for missing metacritic

Testing environment

In [10]:
test = result.json()
print(result.headers['X-Total-Page-Count']) # Important if we want to parse through all pages
print(result.url)

test = pd.DataFrame(test)

header = result.headers

15
https://www.cheapshark.com/api/1.0/deals?storeID=1&lowerPrice=25&upperPrice=50&pageNumber=2


## Results

## Discussion

## Acknowledgements